## Naive Bayes Classifier

- Usage: classification
- Generative (probabilistic) model
- Naive: each input variable is independent from each other
- Effectiveness: 
> Despite these rather optimistic assumptions, naive Bayes classifiers often outperform far more sophisticated alternatives. The reasons are related to Figure 6.15: although the individual class density estimates may be biased, this bias might not hurt the posterior probabilities as much, especially near the decision regions. In fact, the problem may be able to withstand considerable bias for the savings in variance such a “naive” assumption earns. (ESL pp.211)

<img src='images/NaiveBayes_effectiveness.png' width='600'>

> In spite of their apparently over-simplified assumptions, naive Bayes classifiers have worked quite well in many real-world situations, <u>famously document classification and spam filtering</u>. They require a small amount of training data to estimate the necessary parameters. ([sklearn documentation](https://scikit-learn.org/stable/modules/naive_bayes.html))

- Efficiency:
> The naive Bayes classifier **can be extremely fast** compared to more sophisticated methods. The decoupling of the class conditional feature distributions means that <u>each distribution can be independently estimated as a one dimensional distribution</u>. This in turn helps to alleviate problems stemming from the curse of dimensionality. ([sklearn documentation](https://scikit-learn.org/stable/modules/naive_bayes.html))


### `Algorithm Overview`
**Input** 
- Training data $T=\{(x_1, y_1),...(x_N,y_N)\}$, where $x_i \in \mathcal{X} \subseteq \mathbb{R}^n$, $x_i=(x_i^{(1)}, x_i^{(2)}, \ldots, x_i^{(n)})$ and $y_i \in \mathcal{y} = {c_1,c_2,...c_K}$ are the classes. And for a feature $x_i^{(j)}$, its possible value set is $x_i^{(j)} \in \{a_{j1}, a_{j2}, ... , a_{jS_j}\}$. For an exmaple, $a_{jl}$ is the value of $j^{th}$ featue may be the $l^{th}$, $l \in \{1,2,3, \ldots, S_j\}$
- Input example $x$

**Output**
- The class that $x$ belongs to.

**Algorithm**
- Goal: maximise the posterior probabilities $\max\limits_{c_k}P(y=c_k|x)$.
    * It's proven that maximising the posterior probabilitie equals to minimising the 0-1 loss. 
- Bayes equation:
$$\max\limits_{c_k}P(y=c_k|x) = \max\limits_{c_k}\frac{P(x|y=c_k)P(y=c_k)}{P(x)}$$
For an exmaple $x$, $P(x)$ is same for any class $c_i$. So the problem of finding the maximum posterior probability equals to $$\max\limits_{c_k}P(x|y=c_k)P(y=c_k)$$
where $P(x|y=c_k)$ is the conditional probability, and $P(y=c_k)$ is the prior probability.

- Naive: strong assumption on the features: given a class, the features $x^{(j)}$ are independent: 
$$P(x)=\prod_j^nP(x^{(j)})$$
$$P(x|y)=P(x^{(1)}|y) \cdot P(x^{(2)}|y) \ldots \cdot P(x^{(n)}|y)=\prod_j^nP(x^{(j)}|y)$$
Under this naive assumption, the problem now is to solve:
\begin{align}
&\max\limits_{c_k}P(x|y=c_k)P(y=c_k) \\ = &\max\limits_{c_k}P(y=c_k)\prod_j^nP(x^{(j)}|y=c_k) \label{eq:goal} \tag{3}
\end{align}

- How to estimate $P(x|y=c_k)$ and $P(y=c_k)$?
    * Estimate $P(y=c_k)$ 
        1. Maximum Likelihood Estimator
            $$P(y=c_k) = \frac{\sum_{i=1}^N I(y_i=c_k)}{N} = \frac{N_y}{N}\label{eq:pp} \tag{2}$$
            $$\quad k=1,2,\ldots,K; \quad i=1,2,\ldots,N$$
        2. Maximum A Posteriori (MAP):same as Equation $\eqref{eq:pp}$
    * Estimate $P(x|y=c_k)$
        1. When the input variable is discrete: maximum likelihood estimator
            \begin{align}
            P(x^{(j)}=a_{jl}|y=c_k) &= \prod_j^nP(x^{(j)}|y=c_k) \\
            &= \prod_j^n\frac{\sum_{i=1}^N I\Big((x_i^{(j)}=a_{jl}) \quad \&\& \quad (y_i=c_k)\Big)}{\sum_{i=1}^N I(y_i=c_k)} \\
            &= \prod_j^n\frac{N_{yj}}{N_y}
            \label{eq:cp} \tag{1}
            \end{align}
            $$j = 1,2,\ldots,n; \quad l=1,2,\ldots,S_j; \quad k=1,2,\ldots,K; \quad i=1,2,\ldots,N$$
        2. when the input varibale is continuous:
            * continuous $\rightarrow$ discrete
            * assume a distribution (e.g., Gaussian), and fit a probability density function (pdf)
               
- **Avoid Numerical Underflow with Log**: the multiplication of many probabilities together can become numerically unstable, especially when the dimension of input features is high. To overcome this problem, it is common to change the calculation from the product of probabilities to the sum of log probabilities: $\log(P(x^{(1)}|y=c_k)) + \log(P(x^{(2)}|y=c_k)) \ldots +  \log(P(x^{(n)}|y=c_k))$

- **Prevent zero probablities in further computations**: smoothing. For a class $y$, $θ_{yi}$ is the probability 
$P(x_i)∣y)$of feature $i$ appearing in a sample belonging to class $y$. 
$$\hat{\theta}_{yi}=\frac{N_{yj}+\alpha}{N_y+\alpha n}$$
The smoothing priors $\alpha \leq 0$ accounts for features not present in the learning samples and prevents zero probabilities in further computations. Setting $\alpha=1$ is called **Laplace smoothing**, while $\alpha < 1$ is called **Lidstone smoothing**.

- Variations: different naive Bayes classifier differ mainly by the assumptions they make regarding the distribution of $P(x^{(j)}|y=c_k)$
    * **Multinomial Naive Bayes** Hi
    
    * **Gaussian Naive Bayes**: The likelihood of the features is assumed to be Gaussian.
        $$P(x^{(j)}|y) = \frac{1}{\sqrt{2\pi (\sigma_y^{(j)})^2}} exp\Big(-\frac{(x_i- \mu_y^{(j)})^2}{2(\sigma_y^{(j)})^2} \Big)$$
        For a class $y$, $\mu_y^{(j)}$,$\sigma_y^{(j)}$ can be estimated as the mean, variance of all the training examples in terms of feature $j$.
        
    * **Complement Naive Bayes**
    
    * **Bernoulli Naive Bayes**
    
- For $c_k$ classes and $n$ input features, navie Bayes computes $c_k$ prior probabilities.  
- For $c_k$ classes and $n$ input features, navie Bayes computes ? conditional probabilities?
    * If all the features are categorical, $c_k \sum_{j=1}^nS_j$ conditional probabilities are computed
    * In general, $c_k*n$ different **probability distributions** must be created and maintained

- **Steps:**
    1. compute conditional probability with Equation $\eqref{eq:cp}$; compute prior probability with Equation $\eqref{eq:pp}$
    2. Given $x$, compute $P(y=c_k)\prod_j^nP(x^{(j)}|y=c_k)$ under each $c_k$
    3. Select the class with the largest posterior probability as the classification for the given instance



11.11问题：
1. 如果input features中既有取值仅有几类的离散变量，又有符合高斯分布的变量，又有符合其他分布的变量，怎么确定对哪个类别用哪类方法估计$p(x|y=c_k)$？ How to deal with data set containing numeric variables and category variables together? (2019.11.11)
    
    Good question. (Here)[https://stackoverflow.com/questions/14254203/mixing-categorial-and-continuous-data-in-naive-bayes-classifier-using-scikit-lea?rq=1] are two options: 1. transform all to categorical 2. train naive Bayes separately, and then refit a new model based on the predict probabilities.
    
    
2. Gaussian naive Bayes中，如何fit每个输入变量的Gaussian distribution？又如何计算要预测的数据的概率密度？(2019.11.11)

    Answer: For Gaussian distribution, it is determined when two variables: mean and variance are determined. So we need to estimate these two variables. $\mu$ is the sample mean and $\sigma^2$ is the sample variance. Then for an input $x$, we can just calculate the corresponding probability density. (2019.11.12)

### Gaussian naive Bayes

In [17]:
from sklearn import datasets
iris = datasets.load_iris()

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

y_pred = gnb.fit(iris.data, iris.target).predict(iris.data)
print("Number of mislabeled points out of a total %d points : %d"
      % (iris.data.shape[0],(iris.target != y_pred).sum()))

print(gnb.get_params())

print(gnb.theta_)
print(gnb.sigma_)

Number of mislabeled points out of a total 150 points : 6
{'priors': None, 'var_smoothing': 1e-09}
[[5.006 3.428 1.462 0.246]
 [5.936 2.77  4.26  1.326]
 [6.588 2.974 5.552 2.026]]
[[0.121764 0.140816 0.029556 0.010884]
 [0.261104 0.0965   0.2164   0.038324]
 [0.396256 0.101924 0.298496 0.073924]]


**Implementation**

Example:

`X_train = [[1, 'S'], [1, 'M'], [1, 'M'], [1, 'S'],  [1, 'S'],
               [2, 'S'], [2, 'M'], [2, 'M'], [2, 'L'],  [2, 'L'],
               [3, 'L'], [3, 'M'], [3, 'M'], [3, 'L'],  [3, 'L']]`
               
`Y_train = [-1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1] `

In [14]:
import numpy as np
from sklearn import datasets

def normalize(X, axis=-1, order=2):
    """ Normalize the dataset X """
    l2 = np.atleast_1d(np.linalg.norm(X, order, axis))
    l2[l2 == 0] = 1
    return X / np.expand_dims(l2, axis)


def shuffle_data(X, y, seed=None):
    """ Random shuffle of the samples in X and y """
    if seed:
        np.random.seed(seed)
    idx = np.arange(X.shape[0])
    np.random.shuffle(idx)
    return X[idx], y[idx]


def train_test_split(X, y, test_size=0.5, shuffle=True, seed=None):
    """ Split the data into train and test sets """
    if shuffle:
        X, y = shuffle_data(X, y, seed)
    # Split the training data from test data in the ratio specified in
    # test_size
    split_i = len(y) - int(len(y) // (1 / test_size))
    X_train, X_test = X[:split_i], X[split_i:]
    y_train, y_test = y[:split_i], y[split_i:]

    return X_train, X_test, y_train, y_test


def accuracy_score(y_true, y_pred):
    """ Compare y_true to y_pred and return the accuracy """
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy





class NaiveBayes():

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.classes = np.unique(y)
        self.parameters = {}
        for i, c in enumerate(self.classes):
            # 计算每个种类的平均值，方差，先验概率
            X_Index_c = X[np.where(y == c)]
            X_index_c_mean = np.mean(X_Index_c, axis=0, keepdims=True)
            X_index_c_var = np.var(X_Index_c, axis=0, keepdims=True)
            parameters = {"mean": X_index_c_mean, "var": X_index_c_var, "prior": X_Index_c.shape[0] / X.shape[0]}
            self.parameters["class" + str(c)] = parameters

    def _pdf(self, X, classes):
        # 一维高斯分布的概率密度函数
        # eps为防止分母为0
        eps = 1e-4
        mean = self.parameters["class" + str(classes)]["mean"]
        var = self.parameters["class" + str(classes)]["var"]

        # 取对数防止数值溢出
        # numerator.shape = [m_sample,feature]
        numerator = np.exp(-(X - mean) ** 2 / (2 * var + eps))
        denominator = np.sqrt(2 * np.pi * var + eps)

        # 朴素贝叶斯假设(每个特征之间相互独立)
        # P(x1,x2,x3|Y) = P(x1|Y)*P(x2|Y)*P(x3|Y),取对数相乘变为相加
        # result.shape = [m_sample,1]
        result = np.sum(np.log(numerator / denominator), axis=1, keepdims=True)

        return result.T

    def _predict(self, X):
        # 计算每个种类的概率P(Y|x1,x2,x3) =  P(Y)*P(x1|Y)*P(x2|Y)*P(x3|Y)
        output = []
        for y in range(self.classes.shape[0]):
            prior = np.log(self.parameters["class" + str(y)]["prior"])
            posterior = self._pdf(X, y)
            prediction = prior + posterior
            output.append(prediction)
        return output

    def predict(self, X):
        # 取概率最大的类别返回预测值
        output = self._predict(X)
        output = np.reshape(output, (self.classes.shape[0], X.shape[0]))
        prediction = np.argmax(output, axis=0)
        return prediction



#主函数
def main():
    
#     data = datasets.load_digits()
#     X = normalize(data.data)
#     y = data.target

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
#     print("X_train",X_train.shape, "; y_train", y_train.shape)
    
#     X_train = np.array([[1, 'S'], [1, 'M'], [1, 'M'], [1, 'S'],  [1, 'S'],
#                [2, 'S'], [2, 'M'], [2, 'M'], [2, 'L'],  [2, 'L'],
#                [3, 'L'], [3, 'M'], [3, 'M'], [3, 'L'],  [3, 'L']])
    X_train = np.array([[1, 1], [1, 2], [1, 2], [1, 1],  [1, 1],
               [2, 1], [2, 2], [2, 2], [2, 3],  [2, 3],
               [3, 3], [3, 2], [3, 2], [3, 3],  [3, 3]])
    y_train = np.array([2, 2, 1, 1, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 0])
    
    X_test = np.array([[1,1]])
    y_test = np.array([2])
    print("X_train",X_train.shape, "; y_train", y_train.shape)
    
    clf = NaiveBayes()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print ("Accuracy:", accuracy)

    # Reduce dimension to two using PCA and plot the results
#     Plot().plot_in_2d(X_test, y_pred, title="Naive Bayes", accuracy=accuracy, legend_labels=data.target_names)

if __name__ == "__main__":
    main()

X_train (15, 2) ; y_train (15,)
Accuracy: 1.0


### Bernoulli Naive Bayes
Example: [Using navie Bayes for text classification](https://github.com/szcf-weiya/ESL-CN/blob/master/code/NaiveBayes/python/bayes.py):

Inputs are one-hot vector

Classes must be 0 or 1

In [24]:
import numpy as np

def loadDataSet():
    postingList = [['my', 'my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1] #1
    return postingList, classVec


def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # s|t is s.union(t)
    return list(vocabSet)

# word occurrence vectors (1: occur 0: doesn't occur)
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: 
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec   
 
    
def trainNB0(trainMatrix, trainCategory):
    # N examples
    numTrainDocs = len(trainMatrix)
    # n features
    numWords = len(trainMatrix[0])
    
    pAbusive = sum(trainCategory)/float(numTrainDocs)  # why use sum? classes can only be 0 and 1?
    
    #p0Num = np.zeros(numWords); p1Num = np.zeros(numWords)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)
    
    #p0Denom = 0.0; p1Denom = 0.0
    p0Denom = 2.0; p1Denom = 2.0
    
    for i in range(numTrainDocs):
        if (trainCategory[i] == 1):
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    
    #p1Vect = p1Num/p1Denom
    #p0Vect = p0Num/p0Denom
    p1Vect = np.log(p1Num/p1Denom)  # p(x^{(j)}/y=1)
    p0Vect = np.log(p0Num/p0Denom)  # p(x^{(j)}/y=0)
    return p0Vect, p1Vect, pAbusive

def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

if __name__ =="__main__":
    
    listOPosts,listClasses = loadDataSet()
    
    myVocabList = createVocabList(listOPosts)
    print('--- vocabulary list is: --- \n', len(myVocabList))
    
    # prepare training data
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    print('---Each doc to vector: --- \n',trainMat)
    print(np.array(trainMat).shape)
    
    p0V,p1V,pAb=trainNB0(trainMat,listClasses)
    print(len(p0V))
    print(len(p1V))
    print(pAb)

--- vocabulary list is: --- 
 32
---Each doc to vector: --- 
 [[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]]
(6, 32)
32
32
0.5
